# Features 

En este notebook se irán generando distintos features que luego deberán ser probados.

In [1]:
import pandas as pd
import numpy as np
#from sklearn import

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df['id'] = df['id'].astype(np.uint16)
df['target'] = df['target'].astype(np.uint8)
#df = df.drop(labels='location', axis=1)
df = df.fillna("")

## Features a partir de la keyword 

### Count Encoding de las keywords 

In [4]:
keywords = df["keyword"].value_counts()

In [14]:
df["keyword_count"] = df["keyword"].transform(lambda x: keywords[x])

In [15]:
df["keyword_count_norm"] = df["keyword_count"]/keywords.max()

### Mean Encoding de las keywords 

In [17]:
keywords_target = df.groupby(["keyword", "target"]).count()["id"]

In [21]:
df["keyword_mean"] = df.agg(lambda x: keywords_target[x["keyword"]][x["target"]], axis=1)

## Features a partir del tweet 

In [42]:
def hay_nros(x):
    x = x.split()
    for i in x:
        try:
            float(i)
            return 1
        except ValueError:
            pass
    return 0 

df["hay_nros"] = df["text"].transform(hay_nros) #No reconoce los nros con comas